Řešení Poissonovy rovnice metodou konečných diferencí
==============================================================

1D příklady
-----------

Nulové Dirichletovy okr. p.
$$
\left[\begin{smallmatrix}{}
           -2 & 1 &   &  &\\\
            1 & -2& 1 &  &\\\
              &  .& . & .&\\\
              & &  1& -2 & 1\\\
              & &   & 1 & -2
    \end{smallmatrix}\right]
    \cdot
    \left[\begin{smallmatrix}{}
            U_1\\\
            U_2\\\
              \vdots\\\
              U_{n-1}\\\
              U_n
    \end{smallmatrix}\right]
    =-\frac{h^2}{\epsilon_0}
        \left[\begin{smallmatrix}{}
            \rho_1\\\
            \rho_2\\\
              \vdots\\\
              \rho_{n-1}\\\
              \rho_n
    \end{smallmatrix}\right]
$$

In [ ]:
%matplotlib inline
from pylab import *
from numpy import *

Definice problému: Konstantní nábojová hustota s Dirichletovou OP. Síť se 100 body, krok $h=1$, $\epsilon_0=1$. Maximální chyba iteračních metod $10^{-3}$

In [ ]:
eps0 = 1
dx = 1
imax = 100
M = imax
rho = ones(imax)
b = -rho/eps0
epsilon = 1e-3

Iterativní řešení Jacobiho methodou

In [ ]:
%%timeit -n2 -r3
U = zeros(imax+2)
iter = 0
while True:
    res = 0.5*(U[:-2] + U[2:] - dx**2*b) - U[1:-1]
    U[1:-1] += res
    iter += 1
    if sum(res**2) < epsilon**2: break
print(iter)

Iterativní řešení Gauss Seidelovou methodou. Nelze v pythonu jednoduše vektorizovat - používáme cyklus, což ji zpomaluje.

In [ ]:
%%timeit -n2 -r3
U = zeros(imax+2)
iter = 0
while True:
    err=0
    for i in range(1,imax+1):
        res = 0.5*(U[i-1] + U[i+1] - dx**2*b[i-1]) - U[i]
        U[i] += res
        err += res**2
    iter += 1
    if err < epsilon**2: break
print(iter)

Superrelaxace (Successive overrelaxation (SOR)) s faktorem $\omega$, bez vektorizace

In [ ]:
%%timeit -n2 -r3
U = zeros(imax+2)
omega = 2/(1+sqrt(1-cos(pi/imax)**2))

iter = 0
while True:
    err=0
    for i in range(1,imax+1):
        res = 0.5*(U[i-1] + U[i+1] - dx**2*b[i-1]) - U[i]
        U[i] += omega*res
        err += res**2
    iter += 1
    if err < epsilon**2: break
print(iter)

Výše uvedené metody lze vektorizovat iterace zvlášť přes liché a sudé elementy v šachovnicovitém vzoru (red-black ordering)

In [ ]:
%%timeit -n2 -r3
U = zeros(imax+2)
omega = 2/(1+sqrt(1-cos(pi/imax)**2))
iter = 0
while True:
    err=0
    res = 0.5*(U[:-2:2] + U[2::2] - dx**2*b[::2]) - U[1:-1:2]
    U[1:-1:2] += omega*res
    err = sum(res**2)
    
    res = 0.5*(U[1:-2:2] + U[3::2] - dx**2*b[1::2]) - U[2:-1:2]
    U[2:-1:2] += omega*res
    err += sum(res**2)
    
    iter += 1
    if err < epsilon**2: break

### Fourier method

In [ ]:
from scipy.fftpack import dst, idst

In [ ]:
%%timeit
#%%prun
rho_k = dst(b, type=1) # type 1 assumes odd func around n=-1 and n=imax
k = arange(1, M+1)
lambda_k = 2*cos(k*pi/(M+1))-2
U_k = rho_k/lambda_k
U = idst(U_k, type=1)/(2*(M+1))

In [ ]:
k = arange(1, M+1)
lambda_k = 2*cos(k*pi/(M+1))-2

In [ ]:
%%timeit
rho_k = dst(b, type=1) # type 1 assumes odd func around n=-1 and n=imax
U_k = rho_k/lambda_k
U = idst(U_k, type=1)/(2*(M+1))

In 1D, the solution can be obtained using the fast and simple Thomas algorithm. Instead of it we may use a general sparse solver, which is also easily generalized to 2D.

### Direct solver

In [ ]:
from scipy.sparse import dia_matrix, eye
from scipy.sparse.linalg import spsolve
imax = 100
def d2matrix(nelem):
    elements = ones((3,nelem))
    elements[1,:] *= -2
    return dia_matrix((elements, [-1,0,1]), shape=(nelem,nelem)).tocsc()
d2x = d2matrix(imax)/dx**2
b = -ones(imax)

The d2matrix is a matrix of second differences in sparse format:

In [ ]:
print(d2matrix(10).todense())

In [ ]:
%%timeit
#%%prun 
U = spsolve(d2x, b)

Try direct solver with separate factorization

In [ ]:
from scipy.sparse.linalg import factorized
LU = factorized(d2x)

In [ ]:
%%timeit
U = LU(b)

We have obtained a solver, which is more than 10000 faster than our initial attempts

In [ ]:
U = LU(b)
plot(U)